### start

In [7]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import gc
import time
import copy
import logging
import re
from itertools import chain,combinations
import pdb
import math
from pathlib import Path
import pickle
import cloudpickle
import datetime
from datetime import datetime,timedelta
import json
import warnings
import yaml
from typing import Dict,Union,List,Any,Tuple
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import IPython
import requests
import pandas_gbq
from google.cloud import bigquery
from dreams_core.googlecloud import GoogleCloud as dgc
from dreams_core import core as dc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import progressbar
import warnings
import boto3

# load_dotenv(Path("../../../Local/.env"))

# Custom format function for displaying |numbers/
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

# Suppress warnings
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
warnings.filterwarnings('ignore', message='.*frozen modules.*')
warnings.filterwarnings("ignore", message="MallocStackLogging")

# silence pygame donation request
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
os.environ['LOGGING_FILE'] = "../../../Local/logs/wallet_modeling.log"
os.environ['NOTIFICATION_SOUNDS_DIR'] = "../../../Local"

# Dark mode charts
plt.rcParams['figure.facecolor'] = '#181818'  # Custom background color (dark gray in this case)
plt.rcParams['axes.facecolor'] = '#181818'
plt.rcParams['text.color'] = '#afc6ba'
plt.rcParams['axes.labelcolor'] = '#afc6ba'
plt.rcParams['xtick.color'] = '#afc6ba'
plt.rcParams['ytick.color'] = '#afc6ba'
plt.rcParams['axes.titlecolor'] = '#afc6ba'
matplotlib.rcParams['text.usetex'] = False
matplotlib.rcParams['mathtext.default'] = 'regular'

import utils as u
# Set the custom error handler
ipython = IPython.get_ipython()
ipython.set_custom_exc((Exception,), u.notify_on_failure)

player = u.AmbientPlayer()
player.stop_all_players()

# configure logger
logger = u.setup_notebook_logger('../logs/notebook_logs.log')
logger.setLevel(logging.INFO)



# import local modules
# pyright: reportMissingImports=false
sys.path.append('..//src')

import etls.s3_pipeline as sp
import sagemaker_wallets.wallet_modeler as wm
import sagemaker_wallets.workflow_orchestrator as wwo
from utils import ConfigError
import utils as u

# reload all modules
modules = [
    sp,
    wm, wwo,
    u
]

# load all configs

u.export_code(
    code_directories=[
        # 'etls',
        'sagemaker_wallets',
    ],
    # include_config = True,
    # ipynb_notebook = 'DDA-769 coin model score dist toggle.ipynb'
)

[importlib.reload(module) for module in modules]
u.notify('retro')

logger.milestone("Good morning, let's get to work")

[23/Jun/25 17:32:09] INFO [utils.export_code:1494] Consolidation complete. All files are saved in temp/consolidated_code.py
[23/Jun/25 17:32:09] MILESTONE [3510321530.<module>:116] Good morning, let's get to work


# Initial ETLs

In [ ]:
[importlib.reload(module) for module in modules]

sp.upload_folder_to_s3(
    local_path='../s3_uploads/wallet_training_data_queue/dda_861_wallet_graphs_30d',
    bucket_name='wallet-training-data',
    s3_target_folder='imports_raw'
)

[23/Jun/25 15:24:13] INFO [s3_pipeline.upload_folder_to_s3:63] Ready to upload 208 files with total size 0.84GB
[23/Jun/25 15:24:13] INFO [s3_pipeline.upload_folder_to_s3:64] Target: s3://wallet-training-data/imports_raw/dda_861_wallet_graphs_30d
[23/Jun/25 15:24:15] INFO [s3_pipeline.upload_folder_to_s3:82] Uploading y_train_230710.parquet -> s3://wallet-training-data/imports_raw/dda_861_wallet_graphs_30d/y_train_230710.parquet
[23/Jun/25 15:24:16] INFO [s3_pipeline.upload_folder_to_s3:82] Uploading x_eval_240306.parquet -> s3://wallet-training-data/imports_raw/dda_861_wallet_graphs_30d/x_eval_240306.parquet
[23/Jun/25 15:24:18] INFO [s3_pipeline.upload_folder_to_s3:82] Uploading y_test_230710.parquet -> s3://wallet-training-data/imports_raw/dda_861_wallet_graphs_30d/y_test_230710.parquet
[23/Jun/25 15:24:18] INFO [s3_pipeline.upload_folder_to_s3:82] Uploading x_eval_230908.parquet -> s3://wallet-training-data/imports_raw/dda_861_wallet_graphs_30d/x_eval_230908.parquet
[23/Jun/25 15:2

# Code begins

#### initiate orchestrator

In [ ]:
[importlib.reload(module) for module in modules]


workflow_orch = wwo.WalletWorkflowOrchestrator()

In [9]:
data_folder = Path('../s3_uploads') / 'wallet_training_data_queue' / 'dda_861_wallet_graphs_30d'
date_suffixes = [
    '250130',
    '250301'
]
workflow_orch.load_training_data(data_folder,date_suffixes)

#### dda 

In [ ]:
modeler = wm.WalletModeler()
modeler.set_training_data(workflow_orch.training_data)

